In [1]:
num_rows_batch_publish=5000 # number of objects to be batch published to Scorpio



        entity_obj = Entity(entity_type, entity_id_prefix + sensor_name + ":ID:" + str(int(u)) ) # initialize the entity for the new station


In [2]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")
import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response




In [3]:
import pandas as pd
import geopandas as gpd

#gdf_traw_traffic = pd.read_pickle('./../cgarrido/madrid_data/traffic/gdf_raw_traffic_month12.pkl')

In [4]:
magnitudes = ["co", "pm10", "pm2.5", "no2", "o3"] # to be used
gdf_property_list=["co", "pm10", "pm2_5", "no2",  "o3"]
entity_id_prefix =  "urn:ngsi-ld:AirQualityObserved:Berlin"
entity_type = 'AirQualityObserved'


In [5]:
pub_df = {}
gdf_no2 = pd.read_pickle('./../cgarrido/berlin_data/air_pollution/gdf_with_hex_no2.pkl')
gdf_o3 = pd.read_pickle('./../cgarrido/berlin_data/air_pollution/gdf_with_hex_o3.pkl')
gdf_pm10 = pd.read_pickle('./../cgarrido/berlin_data/air_pollution/gdf_with_hex_pm10.pkl')
gdf_pm2_5 = pd.read_pickle('./../cgarrido/berlin_data/air_pollution/gdf_with_hex_pm2.5.pkl')
gdf_co = pd.read_pickle('./../cgarrido/berlin_data/air_pollution/gdf_with_hex_co.pkl')

gdf_no2['data'] = gdf_no2['data'].astype(float)
gdf_o3['data'] = gdf_o3['data'].astype(float)
gdf_pm10['data'] = gdf_pm10['data'].astype(float)
gdf_pm2_5['data'] = gdf_pm2_5['data'].astype(float)
gdf_co['data'] = gdf_co['data'].astype(float)




In [6]:
berlin_hexagons_gdf=  pd.read_pickle('./../cgarrido/berlin_data/hex_grid/hexagons.pkl')
berlin_hexagons_gdf

,hexagon_id,hexagon_x,hexagon_y,geometry,lat,lon
0,0,0,0,"POLYGON ((1490671.563 6896601.483, 1490769.010...",52.525749,13.409063
1,1,1,1,"POLYGON ((1486732.502 6896205.302, 1486830.610...",52.523589,13.373677
2,2,0,1,"POLYGON ((1488848.594 6893262.031, 1488946.321...",52.507496,13.392682
3,3,1,0,"POLYGON ((1488555.212 6899545.594, 1488653.039...",52.541839,13.390056
4,4,-1,-1,"POLYGON ((1494611.363 6896995.788, 1494708.149...",52.527898,13.444455
...,...,...,...,...,...,...
266,266,-9,-7,"POLYGON ((1522486.772 6893415.233, 1522578.827...",52.508285,13.694856
267,267,-9,-9,"POLYGON ((1526156.027 6900082.526, 1526247.513...",52.544718,13.727827
268,268,-7,-9,"POLYGON ((1521933.780 6905992.816, 1522026.023...",52.577001,13.689907
269,269,-8,-9,"POLYGON ((1524045.347 6903037.871, 1524137.212...",52.560864,13.708871


In [7]:
import geojson    
import pandas as pd
from geojson import Polygon
from pyproj import Transformer





transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326") 

def transform_polygon(xx, yy):
    points=[]
    for i in range(len(xx)): # for all points x,y
        result = transformer.transform( xx[i] , yy[i])
        result = (result[1], result[0])
        points.append(result)
    polygon = Polygon([points])  
    return polygon



In [8]:

gdf_airquality_datasets= [gdf_co, gdf_pm10, gdf_pm2_5, gdf_no2,  gdf_o3 ]
gdf_property_list=["CO","PM10",  "PM2_5",  "NO2", "O3"]

gdf_no2

,datetime,station_id,no2,date,geometry,longitude,latitude,data,hexagon_id,hexagon_x,hexagon_y,lat,lon
0,01.01.2022 00:00,117 Schildhornstraße,9,2022-01-01,POINT (1482578.185 6884388.306),13.318226,52.463600,9.0,49,1,4,52.468799,13.324601
1,01.01.2022 00:00,282 Karlshorst,10,2022-01-01,POINT (1506097.582 6888369.949),13.529505,52.485387,10.0,86,-5,-2,52.481679,13.536793
2,01.01.2022 00:00,145 Frohnau,4,2022-01-01,POINT (1480238.022 6918988.234),13.297204,52.652563,4.0,198,7,-1,52.656445,13.292222
3,01.01.2022 00:00,018 Schöneberg,6,2022-01-01,POINT (1485935.933 6888409.675),13.348390,52.485604,6.0,25,1,3,52.487069,13.340950
4,01.01.2022 00:00,085 Friedrichshagen,7,2022-01-01,POINT (1519175.406 6881885.374),13.646985,52.449899,7.0,236,-9,-4,52.453587,13.645474
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157657,31.12.2022 23:00,282 Karlshorst,8,2022-12-31,POINT (1506097.582 6888369.949),13.529505,52.485387,8.0,86,-5,-2,52.481679,13.536793
157658,31.12.2022 23:00,032 Grunewald,5,2022-12-31,POINT (1472195.462 6886120.669),13.224957,52.473080,5.0,111,4,6,52.480502,13.234906
157659,31.12.2022 23:00,115 Hardenbergplatz,10,2022-12-31,POINT (1484208.263 6892238.201),13.332870,52.506543,10.0,26,2,3,52.503158,13.321942
157660,31.12.2022 23:00,221 Karl-Marx-Straße,13,2022-12-31,POINT (1495891.174 6887185.481),13.437819,52.478907,13.0,20,-2,1,52.475286,13.430667



from dateutil import parser

for g in gdf_airquality_datasets:
    g['hour'] = parser.parse(g['datetime']).hour
    print(g)

In [9]:


entity_type_prefix = 'CityLiveabilityIndex:Berlin:SubIndex:AirQualityAnnualMean'
index_list=[]
for g in gdf_property_list:
    index_list.append( 'urn:ngsi-ld:' + entity_type_prefix + ':' + g)

index_list

['urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubIndex:AirQualityAnnualMean:CO',
 'urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubIndex:AirQualityAnnualMean:PM10',
 'urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubIndex:AirQualityAnnualMean:PM2_5',
 'urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubIndex:AirQualityAnnualMean:NO2',
 'urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubIndex:AirQualityAnnualMean:O3']

In [10]:
print(gdf_co.hexagon_id.unique())




[49 13]


In [11]:


def create_sensor_id_list(sensor_id_list, sensor_name): # sensor_name: e.g., NO2, O3, ...
    return_list=[]
    for sensor_id in sensor_id_list:
        return_list.append(entity_id_prefix +   sensor_name   + ":ID:" + str(sensor_id)) 
    return return_list
    

In [12]:
import time

from dateutil import parser

#indices_df_list=[]
counter=0
df_list = []


for gdf in gdf_airquality_datasets: # do the index calculation for all air quality metrics
    index_dict =[]
    relevant_sensor_data= gdf
    
    # check all available hexagons
    available_hexagons= gdf.hexagon_id.unique()
    print('Available hexagons for the air quality dataset: ' +  str(available_hexagons))
    for hexagon_id in available_hexagons:
        #check all buildings inside the hexagon
        hexagon_geometry =berlin_hexagons_gdf[berlin_hexagons_gdf['hexagon_id'] == hexagon_id]['geometry'].iloc[0]
        
        hexagon_data = gdf[gdf['hexagon_id'] == hexagon_id] # yearly data of a particular hexagon where air quality data exists
        
        hexagon_sensor_data = relevant_sensor_data[relevant_sensor_data['hexagon_id'] == hexagon_id]
        hexagon_sensor_id_list= hexagon_sensor_data['station_id'].unique()
        
        related_sensor_entityid_list= create_sensor_id_list(hexagon_sensor_id_list, gdf_property_list[counter])


        all_days_of_year= hexagon_data.datetime.unique() 
        # The annual moving average takes toooo much time. So we do only one annual mean data per hexagon per day.
        for date_time in all_days_of_year:
            
            if(not(parser.parse(date_time).hour ==0 and parser.parse(date_time).minute ==0)):
                continue
            
            #tic = time.perf_counter()
            data_point= hexagon_data[hexagon_data['datetime'] == date_time].iloc[0]
            #print(data_point)
            #print('Date time: ' + str(date_time))
            data_until_date_time = hexagon_data[hexagon_data['datetime'] < date_time]
            current_annual_mean= data_until_date_time['data'].mean() # annual average for the hexagon for the air quality. This applies up until the current time of the year (the number changes through the year)
            #print('Current annual average ' + str(current_annual_average))
            
            
            
                        
            xx, yy = hexagon_geometry.exterior.coords.xy
            transformed_polygon = transform_polygon(xx,yy)

            
            #x = data_point['geometry'].x
            #y= data_point['geometry'].y
            #transformed_point = transformer.transform(y, x )

            
            d={'related_sensors': related_sensor_entityid_list, 'current_annual_mean' :current_annual_mean, 'index_type': index_list[counter],'hexagon_id':hexagon_id, 'date_time': date_time,'geometry': transformed_polygon}
            #print(d)
            
            index_dict.append(d)
            #toc = time.perf_counter()
            #print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")


    index_df = pd.DataFrame(index_dict)
    index_df = index_df.dropna() # drop the values that don`t exist (included in the table as NA)
    index_df.to_pickle("./data/index_berlin_" + str(counter)+ ".pkl")
    df_list.append(index_df)

    #indices_df_list.append(index_df)
                                       
    counter+=1
    



Available hexagons for the air quality dataset: [49 13]
Available hexagons for the air quality dataset: [147 111   2  10  49 236  41  20  14  13  61   5]
Available hexagons for the air quality dataset: [147 111   2  10  49 236  41  20  14  13  61   5]
Available hexagons for the air quality dataset: [ 49  86 198  25 236  10   5 111  20  13  61  14  46  26  41   2 147 139]
Available hexagons for the air quality dataset: [ 13  14 139  10  26 236 147 111 198]


In [13]:
df_list=[]
for i in range(5):
    df = pd.read_pickle("./data/index_berlin_" + str(i) + ".pkl")
    df_list.append(df)

df_list[0]

,related_sensors,current_annual_mean,index_type,hexagon_id,date_time,geometry
1,[urn:ngsi-ld:AirQualityObserved:BerlinCO:ID:11...,0.254167,urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubInd...,49,01.02.2022 00:00,"{'type': 'Polygon', 'coordinates': [[[13.30648..."
2,[urn:ngsi-ld:AirQualityObserved:BerlinCO:ID:11...,0.250000,urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubInd...,49,01.03.2022 00:00,"{'type': 'Polygon', 'coordinates': [[[13.30648..."
3,[urn:ngsi-ld:AirQualityObserved:BerlinCO:ID:11...,0.330556,urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubInd...,49,01.04.2022 00:00,"{'type': 'Polygon', 'coordinates': [[[13.30648..."
4,[urn:ngsi-ld:AirQualityObserved:BerlinCO:ID:11...,0.302083,urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubInd...,49,01.05.2022 00:00,"{'type': 'Polygon', 'coordinates': [[[13.30648..."
5,[urn:ngsi-ld:AirQualityObserved:BerlinCO:ID:11...,0.284167,urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubInd...,49,01.06.2022 00:00,"{'type': 'Polygon', 'coordinates': [[[13.30648..."
...,...,...,...,...,...,...
725,[urn:ngsi-ld:AirQualityObserved:BerlinCO:ID:17...,0.297903,urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubInd...,13,31.05.2022 00:00,"{'type': 'Polygon', 'coordinates': [[[13.44532..."
726,[urn:ngsi-ld:AirQualityObserved:BerlinCO:ID:17...,0.297724,urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubInd...,13,31.07.2022 00:00,"{'type': 'Polygon', 'coordinates': [[[13.44532..."
727,[urn:ngsi-ld:AirQualityObserved:BerlinCO:ID:17...,0.297604,urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubInd...,13,31.08.2022 00:00,"{'type': 'Polygon', 'coordinates': [[[13.44532..."
728,[urn:ngsi-ld:AirQualityObserved:BerlinCO:ID:17...,0.297254,urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubInd...,13,31.10.2022 00:00,"{'type': 'Polygon', 'coordinates': [[[13.44532..."


from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326") 

def transform_polygon(xx, yy):
    points=[]
    for i in range(len(xx)): # for all points x,y
        result = transformer.transform(yy[i], xx[i] )
        points.append(result)
    
    return points




poly = df['geometry'].iloc[0]

print(poly)
xx, yy = poly.exterior.coords.xy
points = transform_polygon(xx,yy)
print(points)


In [20]:
import time

example_flag=False
for entity_index, tmp_gdf in enumerate(df_list): # for each dataset 
    ingestion_starttime= time.perf_counter() 
    example_flag=True

    hexagon_id_list = tmp_gdf.hexagon_id.unique() 
    print("Info: Unique hexagons for the dataset : "+ str(hexagon_id_list))
    feature_list= tmp_gdf.columns.tolist()
    dataset_length = len(tmp_gdf)
    for hexagon_id in hexagon_id_list:
        hexagon_gdf = tmp_gdf[tmp_gdf['hexagon_id']== hexagon_id]
        
        counter =0
        for index, row in hexagon_gdf.iterrows():
            if counter %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
                entity_obj = Entity(entity_type, index_list[entity_index] +  ':Hexagon:' + str(hexagon_id), ["https://raw.githubusercontent.com/smart-data-models/data-models/master/context.jsonld"]) # (re-)initialize the entity
            for f in range(len(feature_list)):
                if(feature_list[f] == 'geometry'): ## GEOPROPERY
                    ### from geojson.geometry import Point, Polygon, MultiLineString
                    entity_obj.add_geoprop("location", row[f], observedAt= datetime.strptime(str(row.date_time), '%d.%m.%Y %H:%M'))
                    #entity_obj.add_prop('location', row[f])  
                elif(feature_list[f]  == 'current_annual_mean'):
                    entity_obj.add_prop('data', row[f], observedAt= datetime.strptime(str(row.date_time), '%d.%m.%Y %H:%M')) ## ObservedAt add also along with the properties
                elif(feature_list[f]  == 'related_sensors'):
                    ### add_rel(name, object, observedAt)
                    related_sensors = row[f]
                    for r in related_sensors:
                        entity_obj.add_rel('providedBy', r, observedAt= datetime.strptime(str(row.date_time), '%d.%m.%Y %H:%M'))
                elif(feature_list[f] not in ['date_time','type']):
                    entity_obj.add_prop(feature_list[f], row[f] , observedAt= datetime.strptime(str(row.date_time), '%d.%m.%Y %H:%M'))                 
            if example_flag:
                print('INFO: Example data from the dataset to be published - as an example on how data looks.')
                print(entity_obj)
                example_flag=False

            if (counter %num_rows_batch_publish==0 or counter == len(hexagon_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
                publish(entity_obj.to_dict()) # publish the entity
            counter+=1 
        print("... published data for hexagon: " +  str(hexagon_id))

    ingestion_endtime= time.perf_counter() 
    print('Info: Dataset is published successfully with entity ID: ' + str(index_list[entity_index]) 
          + ' --- It took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds to publish the dataset with length ' + str(dataset_length))

Info: Unique hexagons for the dataset : [49 13]
INFO: Example data from the dataset to be published - as an example on how data looks.
{'id': 'urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubIndex:AirQualityAnnualMean:CO:Hexagon:49', 'type': 'AirQualityObserved', '@context': ['https://raw.githubusercontent.com/smart-data-models/data-models/master/context.jsonld'], 'providedBy': {'type': 'Relationship', 'object': 'urn:ngsi-ld:AirQualityObserved:BerlinCO:ID:117 Schildhornstraße', 'observedAt': '2022-02-01T00:00:00Z'}, 'data': {'type': 'Property', 'value': 0.25416666666666665, 'observedAt': '2022-02-01T00:00:00Z'}, 'index_type': {'type': 'Property', 'value': 'urn:ngsi-ld:CityLiveabilityIndex:Berlin:SubIndex:AirQualityAnnualMean:CO', 'observedAt': '2022-02-01T00:00:00Z'}, 'hexagon_id': {'type': 'Property', 'value': 49, 'observedAt': '2022-02-01T00:00:00Z'}, 'location': {'type': 'GeoProperty', 'value': {"coordinates": [[[13.306484, 52.473437], [13.307372, 52.461981], [13.325486, 52.457342], [13.

KeyboardInterrupt: 